##### 导入库

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE
from sklearn.utils import shuffle

import box
import json

##### 选择数据

In [14]:
def get_fea(tit):
    path = 'E:\\desktop\\data\\app\\fea\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data


n='N_all.csv'
p='P_all.csv'
# p='P_ice.csv'
# p='P_her.csv'
p='P_mix.csv'
# p='P_ice1.csv'
way="mix"

##### 处理数据

In [15]:
n_fea = get_fea(n)
p_fea = get_fea(p)


fea_dict = {
    'all':
['fft_imag_6',
 'retvalue',
 'fft_real_5',
 'retindex',
 'alt_stderr_min_50',
 'erbc_num10_focus2',
 'minindex',
 'S_ret',
 'fft_angle_3',
 'fft_real_2',
 'fft_imag_3',
 'F2',
 'dmq',
 'F1',
 'maxindex',
 'fft_abs_22'],

    'ice':
['fft_real_5',
 'lumpiness',
 'minindex',
 'retang',
 'fft_imag_3',
 'maxindex',
 'erbc_num10_focus2',
 'alt_rvalue_min_50',
 'S_ret',
 'F1',
 'alt_stderr_min_50',
 'fft_angle_3',
 'F2',
 'S_down',
 'fft_imag_6',
 'fft_angle_2',
 'retindex',
 'dmq',
 'fft_abs_22',
 'maxvalue'],
    'her':
['fft_imag_6',
 'fft_real_5',
 'retindex',
 'dmq',
 'alt_stderr_min_50',
 'V2',
 'erbc_num10_focus2',
 'minindex',
 'S_ret',
 'fft_angle_3',
 'fft_imag_3',
 'F1',
 'F2',
 'minvalue',
 'maxindex',
 'fft_abs_22'],
    'mix':
['dnp',
 'fft_real_5',
 'S',
 'lumpiness',
 'minindex',
 'retang',
 'fft_imag_3',
 'minvalue',
 'maxindex',
 'erbc_num10_focus2',
 'alt_rvalue_min_50',
 'S_ret',
 'F1',
 'maxang',
 'fft_angle_3',
 'F2',
 'S_down',
 'fft_imag_6',
 'fft_angle_2',
 'retindex',
 'S_low',
 'erbc_num10_focus4',
 'dmq',
 'fft_abs_22'],
}



# fea_dict={
#     "all":[
#   "lumpiness",
#   "fft_imag_3",
#   "fft_real_2",
#   "erbc_num10_focus2",
#   "alt_rvalue_min_50",
#   "S_down",
#   "minvalue",
#   "F1",
#   "fft_angle_3",
#   "V2",
#   "minindex",
#   "alt_stderr_min_50",
#   "maxvalue"
# ],
#     "ice":
#     [
#   "fft_real_5",
#   "fft_imag_3",
#   "maxindex",
#   "fft_real_2",
#   "asoc",
#   "erbc_num10_focus2",
#   "erbc_num10_focus4",
#   "alt_rvalue_min_50",
#   "F2",
#   "S_down",
#   "fft_angle_3",
#   "S",
#   "alt_stderr_max_5",
#   "minindex",
#   "fft_abs_22",
#   "fft_imag_6",
#   "alt_stderr_min_50",
#   "fft_angle_2"
# ],
#     'her':['V1',
#  'level_shift_size',
#  'fft_imag_6',
#  'S_low',
#  'fft_abs_22',
#  'retang',
#  'dmq',
#  'mac',
#  'alt_rvalue_min_50',
#  'fft_real_5',
#  'V2'],
#     'mix':
# ['V_sum',
#  'V1',
#  'level_shift_size',
#  'fft_imag_6',
#  'fft_angle_3',
#  'fft_abs_22',
#  'fft_real_2',
#  'lumpiness',
#  'mac',
#  'alt_rvalue_min_50',
#  'F1',
#  'dmq',
#  'fft_imag_3',
#  'S',
#  'fft_real_5',
#  'V2',
#  'maxang']
# }

fea_list=fea_dict[way]
n_fea=n_fea[fea_list]
p_fea=p_fea[fea_list]

n_fea['target']=[0 for i in range(len(n_fea))]
p_fea['target']=[1 for i in range(len(p_fea))]

np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))


##### 划分数据&分箱

In [16]:
proportion=0.3 #验证集占比
X=np_fea[fea_list]
Y=np_fea[['target']]

#过采样
svmsomte=SVMSMOTE(random_state=0)
X,Y=svmsomte.fit_resample(X,Y)



##### 验证结果

In [17]:
while 1:
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=proportion) #,random_state=0 取消随机
    XY_train=pd.concat([X_train,Y_train],axis=1)
    XY_verify=pd.concat([X_test,Y_test],axis=1)
    n_train=XY_train[XY_train['target']==0]
    p_train=XY_train[XY_train['target']==1]
    n_verify=XY_verify[XY_verify['target']==0]
    p_verify=XY_verify[XY_verify['target']==1]

    #分箱
    config=box.divide(n_train,p_train,fea_list)

    n_jugde=np.sum(box.jugde(n_verify,config))
    p_jugde=np.sum(box.jugde(p_verify,config))

    n_jugdes=np.sum(box.jugde(n_fea,config))
    p_jugdes=np.sum(box.jugde(p_fea,config))
    if p_jugdes/len(p_fea)>0.8 and n_jugdes/len(n_fea)<0.1:
        print("检出率: ",p_jugde,"/",len(p_verify),"  ",p_jugde/len(p_verify))
        print("误判率: ",n_jugde,"/",len(n_verify),"  ",n_jugde/len(n_verify))
        print("精确率: ",len(n_verify)-n_jugde+p_jugde,"/",len(n_verify)+len(p_verify),"  ",(len(n_verify)-n_jugde+p_jugde)/(len(n_verify)+len(p_verify)))

        print('-'*50)
        print("检出率: ",p_jugdes,"/",len(p_fea),"  ",p_jugdes/len(p_fea))
        print("误判率: ",n_jugdes,"/",len(n_fea),"  ",n_jugdes/len(n_fea))
        print("精确率: ",len(n_fea)-n_jugdes+p_jugdes,"/",len(n_fea)+len(p_fea),"  ",(len(n_fea)-n_jugdes+p_jugdes)/(len(n_fea)+len(p_fea)))
        break

检出率:  110 / 144    0.7638888888888888
误判率:  22 / 170    0.12941176470588237
精确率:  258 / 314    0.821656050955414
--------------------------------------------------
检出率:  61 / 75    0.8133333333333334
误判率:  38 / 523    0.07265774378585087
精确率:  546 / 598    0.9130434782608695


##### 保存结果

In [18]:
def default_dump(obj):
    """Convert numpy classes to JSON serializable objects."""
    if isinstance(obj, (np.integer, np.floating, np.bool_)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

config_json=json.dumps(config,default=default_dump)#转化为json格式文件

with open('./config/'+way+'.json','w+') as file:
    file.write(config_json)

print(way,"保存成功")

mix 保存成功
